# Imports

In [ ]:
import pdb
from pathlib import Path

import medloader.dataloader.utils as utils
import medloader.dataloader.config as config
import medloader.dataloader.utils_viz as utils_viz
import medloader.dataloader.tensorflow.augmentations as aug
from medloader.dataloader.tensorflow.dataset import ZipDataset

MAIN_DIR = Path().resolve().parent.absolute()
data_dir = Path(MAIN_DIR).joinpath('data')

%load_ext autoreload
%autoreload 2

# Functions

In [ ]:
def get_dataset_han_miccai2015_3D_grid(data_dir, dir_type
            , mask_type, resampled=False
            , transforms=True, filter=True, debug=False):
    
    from medloader.dataloader.tensorflow.han_miccai2015 import HaNMICCAI2015Dataset

    # Step 1 - Get dataset class
    dataset = HaNMICCAI2015Dataset(data_dir=data_dir, dir_type=dir_type
                    , dimension=3, mask_type=mask_type, resampled=resampled
                    , filter=True
                    , debug=debug)

    # Step 2 - Transforms
    if transforms:
        x_shape_w = dataset.w_grid
        x_shape_h = dataset.h_grid
        x_shape_d = dataset.d_grid
        transforms = [
                aug.NormalizeMinMaxSampler(min_val=config.HU_MIN, max_val=config.HU_MAX, x_shape=(x_shape_h, x_shape_w, x_shape_d,1))
                , aug.Rotate3D()
            ]
        dataset.transforms = transforms

    # Step 3 - Filters (to eliminate/reduce background-only grids)
    if filter:
        dataset.filter = aug.FilterByMask(len(dataset.LABEL_MAP), dataset.SAMPLER_PERC).execute

    # Step 4- Return
    return ZipDataset([dataset])

def get_dataset_han_miccai2015_3D_full(data_dir, dir_type
            , mask_type, resampled=False
            , transforms=False
            , debug=False):
    
    from medloader.dataloader.tensorflow.han_miccai2015 import HaNMICCAI2015Dataset

    # Step 1 - Get dataset class
    dataset = HaNMICCAI2015Dataset(data_dir=data_dir, dir_type=dir_type
                    , dimension=3, mask_type=mask_type, resampled=resampled, grid=False
                    , filter=False
                    , debug=debug)

    # Step 2 - Transforms
    if transforms:
        x_shape_w = dataset.w_grid
        x_shape_h = dataset.h_grid
        x_shape_d = dataset.d_grid
        transforms = [
                aug.NormalizeMinMaxSampler(min_val=config.HU_MIN, max_val=config.HU_MAX, x_shape=(x_shape_h, x_shape_w, x_shape_d,1))
            ]
        dataset.transforms = transforms

    # Step 3 - Return
    return ZipDataset([dataset])

# 1. Main - full volume extractor (for viewing purposes)

In [ ]:
batchsize = 1
dataset3D = get_dataset_han_miccai2015_3D_full(data_dir=data_dir, dir_type='train'
                                            , mask_type=config.MASK_TYPE_COMBINED, resampled=False
                                            , transforms=False)

for (X,Y,meta1,meta2) in dataset3D.generator().batch(batchsize):
    print (' - ', X.shape, Y.shape, meta1.numpy())
    if len(Y.shape) == 4:
        datasets_this = utils.get_dataset_from_zip(meta2, dataset3D)
        utils_viz.viz_3d_mask(Y, datasets_this, meta1, meta2)
        pdb.set_trace()

# 2. Main - grid extractor (for ML purposes - a Tensorflow loader)

In [ ]:
batchsize = 1
dataset3D = get_dataset_han_miccai2015_3D_grid(data_dir=data_dir, dir_type='train'
                                            , mask_type=config.MASK_TYPE_ONEHOT, resampled=False
                                            , filter=True)

try:
    for (X,Y,meta1,meta2) in dataset3D.generator().batch(batchsize):
        print (X.shape, Y.shape, meta1.numpy())
        pbd.set_trace()
except:
    import traceback
    traceback.print_exc()

# 3. Main - grid extractor (for ML benchmarking purposes)

In [ ]:

import medloader.dataloader.utils as utils

batchsize = 1
dataset3D = get_dataset_han_miccai2015_3D_grid(data_dir=data_dir, dir_type='train'
                                            , mask_type=config.MASK_TYPE_ONEHOT, resampled=True
                                            , filter=True, transforms=True)

utils.benchmark(dataset3D.generator().batch(batchsize), model_time=0.3)

# 4. Main - grid extractor (for viewing purposes)

In [ ]:
batchsize = 1
dataset3D = get_dataset_han_miccai2015_3D_grid(data_dir=data_dir, dir_type='train'
                                            , mask_type=config.MASK_TYPE_COMBINED, resampled=False
                                            , filter=True, transforms=False)

for (X,Y,meta1,meta2) in dataset3D.generator().batch(batchsize):
    print (' - ', X.shape, Y.shape, meta1.numpy())
    if len(Y.shape) == 4:
        datasets_this = utils.get_dataset_from_zip(meta2, dataset3D)
        utils_viz.viz_3d_mask(Y, datasets_this, meta1, meta2)
        pdb.set_trace()